In [ ]:
import pandas as pd
from datetime import datetime
import plotly.graph_objs as go
import plotly.express as px

In [ ]:
df = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv')
whole_df = df.copy()

In [ ]:
df = whole_df.copy()

# Prune down the data a bit to make plotting more legible
important_countries = set(df.query('population >= 1000000 and total_deaths >= 3000').iso_code)
print(len(important_countries))
df.query('iso_code in @important_countries', inplace=True)
df.query('total_deaths > 0', inplace=True)

# Smooth things out using a 7-day rolling average
cols = ['total_deaths', 'new_deaths', 'total_deaths_per_million', 'new_deaths_per_million']
df[[f'7day_{c}' for c in cols]] = df[cols].rolling(7).mean()

df

In [ ]:
# Isolate the last bit of data for each country
last_df = pd.merge(df, df.query('new_deaths > 0').groupby('iso_code').date.max().reset_index(), on=['iso_code', 'date'])
last_df

In [ ]:
# Plot
fig1 = px.line(df, x='7day_total_deaths_per_million', y='7day_new_deaths_per_million', line_group='iso_code', color='iso_code', render_mode='svg')
fig2 = px.scatter(last_df, x='7day_total_deaths_per_million', y='7day_new_deaths_per_million', color='iso_code', text='location')
fig = go.Figure(
    data=list(fig1.data) + list(fig2.data),
    layout=go.Layout(
        yaxis=go.layout.YAxis(type='log', title='New deaths per million', range=(-3, 2)),
        xaxis=go.layout.XAxis(title='Total deaths per million'),
        title=f'Normalized Deaths (Updated {datetime.now()})'
    )
)
# fig

In [ ]:
fig.write_html('normalized_deaths.html', include_plotlyjs='cdn', full_html=True)